In [1]:
! pip install openai==0.28

In [28]:
import pandas as pd
import openai
from time import strftime

In [29]:
def prompt(Q):
  tries = 0
  while tries<5:
    try:
      response = openai.ChatCompletion.create(
        model=mmodel,
        messages=Q,
        temperature=0.5,
        #max_tokens=tkn,
        frequency_penalty=0,
        presence_penalty=0,
        request_timeout=199
      )['choices'][0]['message']['content']
      break
    except Exception as e:
      tries = tries+1
      print(f"An error occurred {tries:d}th time: {e}")
  return(Q,response)

In [40]:
dtime = strftime("%Y_%m_%d-%H%M%S")

In [51]:
mmodel = "gpt-3.5-turbo-1106"
openai.api_key = "YOUR_API_KEY"

In [52]:
PROPERTY = 'yield strength'

system = 'You follow instructions carefully, you provide as many rows of the table as possible, at least 50 for every prompt.'

# 2024/2/19: abbrev -> full name
qs = ["Provide me with a list of "+PROPERTY+" values for different high entropy alloys materials. Your response should be a table consisting of 2 columns: material, value. The materials have to be typed as unique chemical compositions consisting of chemical element abbreviations and numbers only (e.g. GaAs, but not Gallium Arsenide). The values have to be single numbers, not ranges. With unit after the numbers. Type out as many different values as you can. Include only High entropy alloys. You are not allowed to type anything else than this table.",
      "Continue expanding this table with new values of "+PROPERTY+" making sure you do not duplicate entries. Type out as many different values as you can. You are not allowed to type anything else than this table."]


In [53]:
filename = PROPERTY.replace(" ", "")+'_'+dtime+'.csv'
print(f"Saving to: {filename} and {filename.replace('csv','txt')}")

Saving to: yieldstrength_2024_04_02-105633.csv and yieldstrength_2024_04_02-105633.txt


In [54]:
sss = [{"role": "system", "content": system}]
tab = []
tab_clean = []
ur = 0
um = 0
i = 0

In [55]:
while True:
  # here i send my first prompt first and then loop over the second one over and
  # over again. You will likely have a different approach to this.
  if i<1:
    qq = qs[0]
  else:
    qq = qs[1]
  # save the first prompt to the conversation
  sss.append({"role": "user", "content": qq})
  # send out the first prompt and receive the response
  sss,ans = prompt(sss)
  # save the response to the conversation
  sss.append({"role": "assistant", "content": ans})

  # we are saving the raw prompts and raw responses, in case we want to analyze
  # or postprocess later
  with open(filename.replace('csv','txt'), 'a') as file:
    print("USER: "+qq, file=file)
    print("GPT : "+ans, file=file)

  # Here we start to grab the data into a nicer structure. For simplicity I had
  # a lot of assumptions. I assume that the word 'value' exists in the header
  # (see first prompt), I remove the header, the separator, and get the rest.
  lines = ans.split('\n')
  if 'value' in lines[0].lower():
    ans = '\n'.join(lines[1:])
  lines = ans.split('\n')
  if '----' in lines[0].lower():
    ans = '\n'.join(lines[1:])
  lines = ans.split('\n')

  # here I try to split the table by columns. I'm assuming that they are
  # separated with |, which is not necessary always the case.
  tab.append(ans)
  try:
    for line in tab[-1].strip().split('\n'):
      tab_clean.append(line.strip('|').split('|'))
  except:
    pass

  # another assumption - only two columns, "material" and "value" (see prompt)
  # some cleanup, converting strings to numbers etc.
  # there is no error handling or edge cases, for example 1.6-1.7 will be removed
  # because it technically is not a number (although it kind of is).
  df = pd.DataFrame(tab_clean, columns=['Material', 'Value'])
  df = df[pd.to_numeric(df['Value'], errors='coerce').notna()]
  df['Value'] = pd.to_numeric(df['Value'])
  df.to_csv(filename, index=False)

  # here I count how many new (non-duplicate) materials we are extracting each
  # time, to monitor progress. We stop if more than 10 iterations or not progress
  if len(df.drop_duplicates()) > ur or df['Material'].nunique() > um:
    ur = len(df.drop_duplicates())
    um = df['Material'].nunique()
    i = i+1
    if i > 10:
      print("Stopping due to 10 iterations exceeded")
      break
  else:
    print("Stopping due to NO PROGRESS")
    break


  print(f"Iteration: {i:3} Generated_rows: {len(lines):3};     TOTAL:  Uniq_rows: {len(df.drop_duplicates()):4d}   Uniq_materials: {df['Material'].nunique():4d}")

  
print(df)

Iteration:   1 Generated_rows: 108;     TOTAL:  Uniq_rows:  108   Uniq_materials:  108
           Material  Value
0       CoCrFeMnNi     655
1         CoCrFeNi     430
2     CoCrFeMnNiAl     800
3       CoCrFeNiAl     600
4     CoCrFeMnNiSi     660
..              ...    ...
185     CoCrFeNiLv    2210
186   CoCrFeMnNiTs    2480
187     CoCrFeNiTs    2230
188   CoCrFeMnNiOg    2500
189     CoCrFeNiOg    2250

[190 rows x 2 columns]


In [66]:
system = "You follow instructions carefully"
qs = "I have provide you a table that consisting of 2 columns: material, yield strength value in MPa., Please check that if the values are correct and modify the dataframe to give a third cholumn that stating \"yes\" or \"no\"\n"
# for index, row in df.iterrows():
#     print(row)
#     break
qq = qs + df.to_string()
sss.append({"role": "user", "content": qq})
  # send out the first prompt and receive the response
sss,ans = prompt(sss)
  # save the response to the conversation
sss.append({"role": "assistant", "content": ans})

print(ans)

|    | Material      |   Value | Correct   |
|---:|:--------------|--------:|:----------|
|  0 | CoCrFeMnNi    |     655 | Yes       |
|  1 | CoCrFeNi      |     430 | Yes       |
|  2 | CoCrFeMnNiAl  |     800 | Yes       |
|  3 | CoCrFeNiAl    |     600 | Yes       |
|  4 | CoCrFeMnNiSi  |     660 | Yes       |
|  5 | CoCrFeNiSi    |     460 | Yes       |
|  6 | CoCrFeMnNiTi  |     680 | Yes       |
|  7 | CoCrFeNiTi    |     470 | Yes       |
|  8 | CoCrFeMnNiV   |     700 | Yes       |
|  9 | CoCrFeNiV     |     480 | Yes       |
| 10 | CoCrFeMnNiNb  |     720 | Yes       |
| 11 | CoCrFeNiNb    |     490 | Yes       |
| 12 | CoCrFeMnNiMo  |     740 | Yes       |
| 13 | CoCrFeNiMo    |     510 | Yes       |
| 14 | CoCrFeMnNiW   |     760 | Yes       |
| 15 | CoCrFeNiW     |     530 | Yes       |
| 16 | CoCrFeMnNiZr  |     780 | Yes       |
| 17 | CoCrFeNiZr    |     550 | Yes       |
| 18 | CoCrFeMnNiHf  |     800 | Yes       |
| 19 | CoCrFeNiHf    |     600 | Yes       |
| 20 | CoC

In [71]:
lines = ans.split('\n')
if 'value' in lines[0].lower():
    ans = '\n'.join(lines[1:])
lines = ans.split('\n')
if '----' in lines[0].lower():
    ans = '\n'.join(lines[1:])
lines = ans.split('\n')
print(lines)


try:
    for line in tab[-1].strip().split('\n'):
      tab_clean.append(line.strip('|').split('|'))
except:
    pass


['|  0 | CoCrFeMnNi    |     655 | Yes       |', '|  1 | CoCrFeNi      |     430 | Yes       |', '|  2 | CoCrFeMnNiAl  |     800 | Yes       |', '|  3 | CoCrFeNiAl    |     600 | Yes       |', '|  4 | CoCrFeMnNiSi  |     660 | Yes       |', '|  5 | CoCrFeNiSi    |     460 | Yes       |', '|  6 | CoCrFeMnNiTi  |     680 | Yes       |', '|  7 | CoCrFeNiTi    |     470 | Yes       |', '|  8 | CoCrFeMnNiV   |     700 | Yes       |', '|  9 | CoCrFeNiV     |     480 | Yes       |', '| 10 | CoCrFeMnNiNb  |     720 | Yes       |', '| 11 | CoCrFeNiNb    |     490 | Yes       |', '| 12 | CoCrFeMnNiMo  |     740 | Yes       |', '| 13 | CoCrFeNiMo    |     510 | Yes       |', '| 14 | CoCrFeMnNiW   |     760 | Yes       |', '| 15 | CoCrFeNiW     |     530 | Yes       |', '| 16 | CoCrFeMnNiZr  |     780 | Yes       |', '| 17 | CoCrFeNiZr    |     550 | Yes       |', '| 18 | CoCrFeMnNiHf  |     800 | Yes       |', '| 19 | CoCrFeNiHf    |     600 | Yes       |', '| 20 | CoCrFeMnNiTa  |     820 | Yes  

In [103]:
clean = []
try:
    for line in ans.strip().split('\n'):
        segments = line.strip('|').split('|')
        clean.append(segments)
except:
    pass

check_df = pd.DataFrame(clean, columns=['index', 'Material', 'Value', 'Correct'])

check_df[check_df['Correct'].str.strip() == 'No']

,index,Material,Value,Correct


In [106]:
# load GTD
gtd_df = pd.read_csv('yield_strength_GTD.csv')
gtd_df

,material,value,passage
0,Al0.5CoCrCuFeNi high entropy alloys,1284.0,Fatigue behavior of Al0.5CoCrCuFeNi high entro...
1,Al0.5CoCrCuFeNi high entropy alloys,1284.0,Fatigue behavior of Al0.5CoCrCuFeNi high entro...
2,Ni2.0 alloy,700.0,Directly cast bulk eutectic and near-eutectic ...
3,Ni2.1,595.0,Directly cast bulk eutectic and near-eutectic ...
4,Ni2.0,580.0,Directly cast bulk eutectic and near-eutectic ...
...,...,...,...
1059,Inconel 718,95.0,The sliding wear behaviour of CoCrFeMnNi and A...
1060,GCr15 steel,2033.9,Investigation into reciprocating dry sliding f...
1061,CoCrFeMnNi HEA,1700.0,Investigation into reciprocating dry sliding f...
1062,CoCrFeMnNi (16 at.%% Mo),765.0,Improved tribological performance of nickel ba...


In [118]:
check_list = list(check_df['Material'])
gen_value = list(check_df['Value'])
gtd_list = list(gtd_df['material'])
gtd_value = list(gtd_df['value'])
total = 0
for m in check_list:
    if m.strip() in gtd_list:
        print(m, gen_value[check_list.index(m)], int(gtd_value[gtd_list.index(m.strip())]))
        total += 1
total

 CoCrFeMnNi          655  605
 CoCrFeNi            430  388
 CoCrFeMnNiV         700  1660


3

In [122]:
likely_hood = []
for m in check_list:
    if m.strip() in gtd_list:
        likely_hood.append(1)
    else:
        material = m.strip()
        maxize = 0
        for gtd in gtd_list:
            for i in range(0, len(material), 2):
                if material[:len(material)-1-i] in gtd:
                    maxize = max(len(material)-i, maxize)
        likely_hood.append(maxize / len(material))
likely_hood

[1,
 1,
 0.8333333333333334,
 1.0,
 0.8333333333333334,
 1.0,
 0.8333333333333334,
 1.0,
 1,
 1.0,
 0.8333333333333334,
 1.0,
 0.8333333333333334,
 1.0,
 1.0,
 1.0,
 0.8333333333333334,
 0.8,
 0.8333333333333334,
 0.8,
 0.8333333333333334,
 1.0,
 0.8333333333333334,
 0.8,
 0.8333333333333334,
 0.8,
 0.8333333333333334,
 0.8,
 0.8333333333333334,
 1.0,
 0.8333333333333334,
 0.8,
 0.8333333333333334,
 1.0,
 0.8333333333333334,
 0.8,
 0.8333333333333334,
 0.8,
 0.8333333333333334,
 0.8,
 0.8333333333333334,
 1.0,
 0.8333333333333334,
 0.8,
 0.8333333333333334,
 0.8,
 0.8333333333333334,
 0.8,
 0.8333333333333334,
 1.0,
 0.8333333333333334,
 1.0,
 0.8333333333333334,
 0.8,
 1.0,
 1.0,
 0.8333333333333334,
 1.0,
 0.8333333333333334,
 0.8,
 0.8333333333333334,
 1.0,
 1.0,
 0.8,
 0.8333333333333334,
 0.8,
 1.0,
 0.8,
 0.8333333333333334,
 0.8,
 0.8333333333333334,
 0.8,
 0.8333333333333334,
 1.0,
 0.8333333333333334,
 1.0,
 0.8333333333333334,
 0.8,
 0.8333333333333334,
 0.8,
 0.8333333333333

In [127]:
sum(likely_hood) / len(likely_hood)

0.8682456140350884